In [ ]:
rush_hour = "昏峰"    # '晨峰' 或 '昏峰'
senerio = "現況"  # '現況' 或 '全線拆除' 或 '部分拆除'
case = ""  # '' 或 '士林北投三重' 或 '全區域'

if case == '':
    case_num = '' 
elif case == '士林北投三重':
    case_num = '_case1'
elif case == '全區域':
    case_num = '_case2'

import pandas as pd
import numpy as np

df = pd.read_excel("新生高替代路線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]

# 處理 Excel 中的空值
sheet1 = sheet1.fillna(0)
sheet1 = sheet1.replace('', 0)

ue_results_1 = pd.read_csv(f"{senerio}_{rush_hour}{case_num}/104_{senerio}_{rush_hour}{case}_UE_results.dat", sep="\t")
# 處理可能的空值
for df_name, df_data in [("ue_results_1", ue_results_1)]:
    df_data['tailNode'] = df_data['tailNode'].fillna(0)
    df_data['headNode'] = df_data['headNode'].fillna(0)
    df_data['UE_flow'] = df_data['UE_flow'].fillna(0)

# 安全地建立 link_name
def safe_link_name(tail, head):
    """安全地建立 link_name，處理空值"""
    try:
        if pd.isna(tail) or pd.isna(head) or tail == 0 or head == 0:
            return "0_0"
        return f"{int(tail)}_{int(head)}"
    except (ValueError, TypeError):
        return "0_0"

ue_results_1["link_name"] = ue_results_1.apply(lambda row: safe_link_name(row['tailNode'], row['headNode']), axis=1)

ue_results_1 = ue_results_1.set_index("link_name")

# 安全地建立 sheet1 的 link_name
def safe_sheet_link_name(a, b):
    """安全地建立 sheet1 的 link_name，處理空值"""
    try:
        if pd.isna(a) or pd.isna(b) or a == 0 or b == 0:
            return "0_0"
        return f"{int(a)}_{int(b)}"
    except (ValueError, TypeError):
        return "0_0"

sheet1["link_name1"] = sheet1.apply(lambda row: safe_sheet_link_name(row['A1'], row['B1']), axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: safe_sheet_link_name(row['A2'], row['B2']), axis=1)

# 安全地合併流量數據，檢查索引是否存在
def safe_add_flow(df, target_link, source_link):
    """安全地合併流量，檢查索引是否存在"""
    try:
        if target_link in df.index and source_link in df.index:
            df.loc[target_link, "UE_flow"] += df.loc[source_link, "UE_flow"]
        else:
            print(f"警告：找不到連結 {target_link} 或 {source_link}")
    except Exception as e:
        print(f"合併流量時發生錯誤：{e}")


# 安全地計算變化百分比
def safe_percentage_change(new_val, old_val):
    """安全地計算百分比變化，處理零值和空值"""
    try:
        if pd.isna(new_val) or pd.isna(old_val) or old_val == 0:
            return 0.0
        return (new_val - old_val) / old_val * 100
    except (ValueError, TypeError, ZeroDivisionError):
        return 0.0

for index, row in sheet1.iterrows():
    try:
        
            
        link_name1 = row['link_name1']
        link_name2 = row['link_name2']
        
        # 檢查連結是否存在於所有結果中
        if (link_name1 not in ue_results_1.index or 
            link_name2 not in ue_results_1.index ) :
            print(f"警告：連結 {link_name1} 或 {link_name2} 在某些結果中不存在，跳過 {row['link']}")
            continue
        
        # 安全地取得流量數據
        flow_1_s = ue_results_1.loc[link_name1, "UE_flow"]
        capacity_1_s = ue_results_1.loc[link_name1, "capacity"]
        vc_1_s = flow_1_s/capacity_1_s
        flow_1_n = ue_results_1.loc[link_name2, "UE_flow"]
        capacity_1_n = ue_results_1.loc[link_name2, "capacity"]
        vc_1_n = flow_1_n/capacity_1_n
        
        if sheet1.loc[index, 'link'] == '重慶北路公車專用道':
            重慶北路公車專用道_s = flow_1_s
            重慶北路公車專用道_n = flow_1_n
            continue
        elif sheet1.loc[index, 'link'] == '重慶北路':
            flow_1_s += 重慶北路公車專用道_s
            flow_1_n += 重慶北路公車專用道_n
            capacity_1_s += 475
            capacity_1_n += 475
            vc_1_s = flow_1_s/capacity_1_s
            vc_1_n = flow_1_n/capacity_1_n
        elif sheet1.loc[index, 'link'] == '中山北路(快車道)':
            中山北路快車道_s = flow_1_s
            中山北路快車道_n = flow_1_n
            continue
        elif sheet1.loc[index, 'link'] == '中山北路(慢車道)':
            row['link'] = '中山北路'
            flow_1_s += 中山北路快車道_s
            flow_1_n += 中山北路快車道_n
            capacity_1_s += 1650
            capacity_1_n += 1650
            vc_1_s = flow_1_s/capacity_1_s
            vc_1_n = flow_1_n/capacity_1_n

        LOS_1_s = "A" if vc_1_s <= 0.25 else "B" if vc_1_s <= 0.5 else "C" if vc_1_s <= 0.8 else "D" if vc_1_s <= 0.9 else "E" if vc_1_s <= 1.0 else "F"
        LOS_1_n = "A" if vc_1_n <= 0.25 else "B" if vc_1_n <= 0.5 else "C" if vc_1_n <= 0.8 else "D" if vc_1_n <= 0.9 else "E" if vc_1_n <= 1.0 else "F"

        print(f'''{row['link']}
        【南向】  流量    {flow_1_s:.2f}    服務水準    {LOS_1_s}
        【北向】  流量    {flow_1_n:.2f}    服務水準    {LOS_1_n}
        ''')
        
    except Exception as e:
        print(f"處理 {row.get('link', 'unknown')} 時發生錯誤：{e}")
        continue